### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config


from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

from sklearn.model_selection import train_test_split


In [2]:
ls data/AD_NMT-master

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [62]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    en_msa_pre_train_ds = pickle.load(handle) 
    
with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    en_msa_test_ds = pickle.load(handle) 
    
with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    lav_msa_pre_train_ds = pickle.load(handle)
    
with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    lav_msa_test_ds = pickle.load(handle)    
    
with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA = pickle.load(handle) 

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA = pickle.load(handle) 
    

In [63]:
print(en_msa_pre_train_ds[0:5])
print(en_msa_test_ds[0:5])

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[["Where's your money?", 'اين مالك؟'], ['Be prepared', 'كن مستعدا'], ["I figured you'd be impressed", 'توقعت انك ستنبهر'], ['May I come in?', 'هل بامكاني الدخول؟'], ['Read through the article', 'اقرا المقالة']]


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [5]:
ls data/AD_NMT-master/

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [105]:
ls data/train

en_formatted_.txt


In [94]:
rm data/train/*

In [160]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa',datatype=''):
    src_formatted = src + '_formatted_' + '.txt'
    trg_formatted = trg + '_formatted_' + '.txt'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as enpy, open(file_path + datatype + "/" + trg_formatted, 'wt') as arpy:
        for i, arr in enumerate(ds):
            print(arr)
            break
            enpy.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            arpy.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [123]:
#pytorch_format(data_LAV_MSA, 'lav', 'msa', '')
#pytorch_format(data_English_MSA, 'en', 'msa', '')
#pytorch_format(data_Magribi_MSA, 'mag', 'msa', '')

In [161]:
#splits the datatsets into train and validation sets 
en_msa_train_ds, en_msa_val_ds = train_test_split(en_msa_pre_train_ds, test_size=.2)
lav_msa_train_ds, lav_msa_val_ds = train_test_split(lav_msa_pre_train_ds, test_size=.2)

In [162]:
print(len(en_msa_train_ds))
print(len(en_msa_val_ds))

7200
1800


In [163]:
pytorch_format(en_msa_train_ds, 'en', 'msa', 'train')
pytorch_format(en_msa_val_ds, 'en', 'msa', 'val')
pytorch_format(en_msa_test_ds, 'en', 'msa', 'test')

pytorch_format(lav_msa_train_ds, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val_ds, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test_ds, 'lav', 'msa', 'test')

['Pus has formed in the wound', 'تكون الصديد في الجرح']
['Do you really love me?', 'اتحبني بحق؟']
["Where's your money?", 'اين مالك؟']
['وفي نا بتتزاعل وحبايب بفترقوا', 'وبعض الناس يغضبون على بعضهم البعض والاعزاء يتركون بعضهم']
['وانسي المفروض واللازم والي بيصير والي ما بصير ', 'انسى المفروض و اللازم و الذى يصلح و الذى لا يصلح']
['بعدو الهاتف شغال يا فؤاد', 'هل لازال الهاتف يعمل يا فؤاد']


In [135]:
ls data/train

en_formatted_.txt   lav_formatted_.txt  msa_formatted_.txt


In [159]:
rm -r data/validation

In [136]:
cat data/train/en_formatted_.txt

TRAIN_000\01\The floor was covered with blood
TRAIN_001\01\How are you Tom?
TRAIN_002\01\"May I come in?" "Yes certainly"
TRAIN_003\01\He tried to get rid of the ants
TRAIN_004\01\You betrayed me
TRAIN_005\01\There's a book here
TRAIN_006\01\All of my friends like soccer
TRAIN_007\01\Does this letter have to be written in English?
TRAIN_008\01\Why did you quit?
TRAIN_009\01\She looked as if she hadn't eaten in days
TRAIN_10\01\I agree with his opinion
TRAIN_11\01\She speaks Polish
TRAIN_12\01\Tom said he saw nothing
TRAIN_13\01\You are not in a position to ask for anything
TRAIN_14\01\I don't know what to do tomorrow
TRAIN_15\01\I think you'll approve
TRAIN_16\01\There is one apple on the desk
TRAIN_17\01\A child could not have understood the situation
TRAIN_18\01\Tom easily convinced Mary that he was innocent
TRAIN_19\01\A big bomb fell and a great many people lost their lives
TRAIN_20\01\How was your vacation?
TRAIN_21\01\No one came except Mary
TRAIN_22\01\He kept a seat for me
TRAI

TRAIN_5441\01\I just arrived now
TRAIN_5442\01\This bracelet is more expensive than that one
TRAIN_5443\01\She was on the verge of crying
TRAIN_5444\01\I've finally finished writing the report
TRAIN_5445\01\What time did she arrive at Narita Airport?
TRAIN_5446\01\What do you think is the best way to settle this conflict?
TRAIN_5447\01\I stood at the end of the line
TRAIN_5448\01\You should work hard
TRAIN_5449\01\I really miss you
TRAIN_5450\01\Show me another example
TRAIN_5451\01\I want to go to Italy
TRAIN_5452\01\He's a tennis player
TRAIN_5453\01\Which diet is better?
TRAIN_5454\01\Stop teasing your sister!
TRAIN_5455\01\The house has three floors
TRAIN_5456\01\This book is about a king who loses his crown
TRAIN_5457\01\Tom deserved the punishment he got
TRAIN_5458\01\I went to the hospital to visit him
TRAIN_5459\01\Does he know what you did?
TRAIN_5460\01\This is a pun
TRAIN_5461\01\Are they still angry?
TRAIN_5462\01\We have to do this again tomorrow
TRAIN_5463\01\Please tell 

In [137]:
cat data/train/msa_formatted_.txt

TRAIN_0000\01\في امان الله
TRAIN_0001\01\و توجد نقطة ملونة يقولون انظري اليها و ركزي فيها ثلاثين ثانية
TRAIN_0002\01\نعم
TRAIN_0003\01\اشكرك يا هاجر و لكن الدموع لا تعني بالضرورة الحزن قد تعني دموع الفرحة و هو اكتر نوع احبه بعد البكاء من خشية الله
TRAIN_0004\01\اننا نغير من الداخل ثم نبدا بالتغيير من الخارج و يكفينا شرف المحاولة
TRAIN_0005\01\عصير بلح و سندوتشات و لا اجد طياره نزلت و خطفتك مثلما ياتى احد و ياخذك لكى تفطر اليس هذا 
TRAIN_0006\01\قالت لي البسي ما شئت لا تعجبينني
TRAIN_0007\01\كنت اخشى ان توافق المناقشة
TRAIN_0008\01\كريمة كيف نتصرف حيال البرنامج ترجمان
TRAIN_0009\01\نعم بقي اشخاص قليلون يدعوهم
TRAIN_010\01\تركيز وتركيز علي النقاط فقط وبسرعة كبيرة وبنظره قليله الحده في اقل من ثواني سوف تجدها وجرب مع واحد جديد عليها
TRAIN_011\01\لكننى احلم بجناحات حريتى
TRAIN_012\01\لالالالالالا   اقصد ان نذاكر جمعيا 
TRAIN_013\01\ليس لها الحق في ذلك لقد تحصلت على ليسانس علوم سياسية نظام قديم
TRAIN_014\01\فعلا  واضح انه يجب اعادة كتابته
TRAIN_015\01\اما الان كل شيء متوفر و نعيش برفاهية و م

TRAIN_2621\01\قد قاربت و لكن عندما يكون عبقرى مثلى يقف امام وغد مثلك
TRAIN_2622\01\ يا جماعة ارايتم هذه النظرة انها تدل على انكم اتخذتكم القرار دون ان تسالوني
TRAIN_2623\01\ ليس ذلك المحل   
TRAIN_2624\01\انا ايضا لم اتغد  اكلنا فاصولياء في المنزل
TRAIN_2625\01\لا يمكن الا تحبه
TRAIN_2626\01\رات امي الاخر يكلف اقل
TRAIN_2627\01\لكن لا امل
TRAIN_2628\01\ قالت لي هذا الموضوع عولج من قبل فما رايك في هذا الكلام   
TRAIN_2629\01\ هنا حتى يكتبوا لي التاريخ   
TRAIN_2630\01\اعتقد انه كان في الحادي عشر
TRAIN_2631\01\لقد احببت ان ارتديها  مع الحذاء
TRAIN_2632\01\ اذهبي فحسب يا رؤى الشاي موجود   
TRAIN_2633\01\ ماذا ستفعلين يجب ان تعتادي   
TRAIN_2634\01\ امي امراتان وحدهما في قرية هل سيتركوننا في سلام    
TRAIN_2635\01\جمال حوريات البحر فى القصص الاساطيرية
TRAIN_2636\01\هل تجتازون الامتحانات في المساء
TRAIN_2637\01\الو
TRAIN_2638\01\انا تاخرت بسبب مشكلة النقل بقيت في انتظار الحافلة ما يقارب الساعة
TRAIN_2639\01\امس تحصلت على منتدى لامثال جزائرية 362 صفحة لا باس
TRAIN_2640\01\انا المتعبة لا هو
T

TRAIN_7462\01\فهمت القصة كان شخصا يروي
TRAIN_7463\01\اين انت يا مراد 
TRAIN_7464\01\انا لا افهم لماذا يحصل التلاميذ الذين يسجلون في حصص الدعم على معدلات ممتازة حتى و ان كانت رؤوسهم فارغة   و الذين لا يسجلون في دروس الدعم و يبدلون مجهودهم كاملا يحصلون على معدلات متواضعة
TRAIN_7465\01\كم اكره اولئك الذين يفعلون ذلك
TRAIN_7466\01\المصرى سيغرق
TRAIN_7467\01\هل يغلي هذا الحليب
TRAIN_7468\01\انا لم اقل شىء
TRAIN_7469\01\ قراتها في الجريدة   
TRAIN_7470\01\سورة الكهف ايضا تجنبك هي ايضا
TRAIN_7471\01\ سفيان اختطفوه   
TRAIN_7472\01\ لكن قصير يا امين كانه معطف قصير و ليس لديه قبعة   
TRAIN_7473\01\اننا كمثل الجالسين في حديقة يبحثون عن موضوع يتحدثون فيه
TRAIN_7474\01\كلا لم يزودوهم  مراد
TRAIN_7475\01\و ماذا عن المحاسبة
TRAIN_7476\01\لقد سعدت لك 
TRAIN_7477\01\لا تصرخي ايتها الخرساء
TRAIN_7478\01\يعرف
TRAIN_7479\01\صحيح
TRAIN_7480\01\قالت لي هل تعلمين بما  ينعتنا
TRAIN_7481\01\اين وصلت في مشروعك الدراسي
TRAIN_7482\01\انه يحب ان تكون لديه عيوب او مشاكل فى شخصيته
TRAIN_7483\01\ حتى ارفع المستوى   

TRAIN_10794\01\لماذا عدت
TRAIN_10795\01\ايضا  المراة و احدث الازياء
TRAIN_10796\01\كي تفهمونني يجب ان اتكلم كثيرا  الكتابة ليست فعالة
TRAIN_10797\01\وهذا فى حالة انه انسان مجرم بكل المقاييس  وهى لا تمتلك اى شئ فى يديها غير هذا لكى تنقذ نفسها من الموت على يديه 
TRAIN_10798\01\ اعطيني فكرة من عندك   
TRAIN_10799\01\ربع ساعة
TRAIN_10800\01\بالطبع  ليكن الله معك  لما لا
TRAIN_10801\01\لقد احببت ان ارتديها  مع الحذاء
TRAIN_10802\01\ساسدله  لا تسريحة و لا هم يحزنون
TRAIN_10803\01\لماذا اتالم بهذا الشكل لماذا
TRAIN_10804\01\ ارايت في المسلسل المعاطف التي تلبسها ياسمين رائعة   
TRAIN_10805\01\احد الاشخاص تشاجر مع زوجته
TRAIN_10806\01\و كل يوم له طعم و لون و رائحة
TRAIN_10807\01\قدر الله وما شاء فعل 
TRAIN_10808\01\ساتصل بزهية و اقول لها اني عرفت ان ابنتك تدرس هنا دعيها تاتي مع البنات الى هنا من  فترة لاخرى
TRAIN_10809\01\رائع  
TRAIN_10810\01\ما ذا قالت ايضا
TRAIN_10811\01\جدا
TRAIN_10812\01\ تريدين ان تشتري حذاء   
TRAIN_10813\01\المرة القادمة
TRAIN_10814\01\انت تسهر تشاهد التليفزيون او تلعب 

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [26]:
len(data_English_MSA)

10001

In [27]:
"""
Create a text file with all the MSA vocab available for SentencePiece to create a library 
"""

text_file_en = open("data/english_data.txt", "wt")
text_file_msa = open("data/arabic_data.txt", "wt")

MSA_text = ""
EN_text = ""

for line in data_English_MSA:
        english = line[0]
        english_words = english.split(" ")
        for count, word in enumerate(english_words):
            text_file_en.write(word)
        text_file_en.write("\n")
        
        arabic = line[1]
        arabic_words = arabic.split(" ")
        for count, word in enumerate(arabic_words):
            text_file_msa.write(word)
        text_file_msa.write("\n")


text_file_en.close()
text_file_msa.close()

In [28]:
spm.SentencePieceTrainer.train('--input=data/arabic_data.txt --model_prefix=data/msa --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/english_data.txt --model_prefix=data/en --vocab_size=2000')

True

In [29]:
sp = spm.SentencePieceProcessor()
sp.load('data/en.model')

True

In [30]:
ls

Env_Setup_Instructions.txt  data/
Model_1.ipynb               transformers/
README.md


In [31]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[89, 83, 12, 83, 8, 83, 6, 309]


## Spacy Tokenizer

In [140]:
ls data/

AD_NMT-master/
AD_NMT-master.zip
PADIC.xml
UNv1.0.ar-en.tar.gz.00
UNv1.0.ar-en.tar.gz.01
ar_formatted_.txt
ar_formatted_test.txt
ar_formatted_train.txt
ar_formatted_validation.txt
arabic-to-english-translation-sentences.zip
arabic_data.txt
en.model
en.vocab
en_formatted_.txt
en_formatted_test.txt
en_formatted_train.txt
english_data.txt
lav_formatted_.txt
mag_formatted_.txt
msa.model
msa.vocab
msa_formatted_.txt
test/
train/
validation/


In [141]:
text_file_en = open("data/en.vocab", "r")
text_file_ar = open("data/msa.vocab", "r")

en_vocab_list = []
ar_vocab_list = []
for line in text_file_en.readlines():
    en_vocab_list.append(line.split("\t")[0])

for line in text_file_ar.readlines():
    print(line)
    print(line.split("\t")[0])
    print("---------")
    ar_vocab_list.append(line.split("\t")[0])

<unk>	0

<unk>
---------
<s>	0

<s>
---------
</s>	0

</s>
---------
ا	-3.13275

ا
---------
▁	-3.1368

▁
---------
ت	-3.45305

ت
---------
ي	-3.56965

ي
---------
ال	-3.57586

ال
---------
؟	-3.73331

؟
---------
ب	-3.99997

ب
---------
م	-4.04006

م
---------
ك	-4.05122

ك
---------
ة	-4.19093

ة
---------
ل	-4.22863

ل
---------
ه	-4.35769

ه
---------
ن	-4.60958

ن
---------
في	-4.72085

في
---------
و	-4.78685

و
---------
ع	-4.84808

ع
---------
▁ا	-4.85412

▁ا
---------
توم	-4.92814

توم
---------
من	-5.00118

من
---------
ها	-5.08404

ها
---------
ف	-5.09055

ف
---------
ان	-5.12223

ان
---------
د	-5.12656

د
---------
ر	-5.13282

ر
---------
ق	-5.13918

ق
---------
نا	-5.25225

نا
---------
ح	-5.31822

ح
---------
ني	-5.32711

ني
---------
ج	-5.3788

ج
---------
ط	-5.38223

ط
---------
ما	-5.40789

ما
---------
ص	-5.43782

ص
---------
خ	-5.44452

خ
---------
ش	-5.55425

ش
---------
ين	-5.66536

ين
---------
ز	-5.68319

ز
---------
ذلك	-5.6844

ذلك
---------
على	-5.69837

على


---------
طوكيو	-8.10943

طوكيو
---------
لعبالتنس	-8.10943

لعبالتنس
---------
الحقيقة	-8.10944

الحقيقة
---------
امريكا	-8.10947

امريكا
---------
مجددا	-8.10948

مجددا
---------
وجود	-8.10974

وجود
---------
▁انالست	-8.10995

▁انالست
---------
▁هلا	-8.11521

▁هلا
---------
طلق	-8.11559

طلق
---------
ركض	-8.11641

ركض
---------
ساعدني	-8.11649

ساعدني
---------
عجب	-8.11806

عجب
---------
الرجل	-8.1221

الرجل
---------
تلك	-8.1233

تلك
---------
▁رايت	-8.12409

▁رايت
---------
مشي	-8.12464

مشي
---------
جلس	-8.1247

جلس
---------
قطع	-8.12482

قطع
---------
قص	-8.12704

قص
---------
الشهر	-8.13303

الشهر
---------
سيكون	-8.13406

سيكون
---------
اليابانية	-8.14205

اليابانية
---------
الناس	-8.14403

الناس
---------
توقع	-8.15165

توقع
---------
يجب	-8.15207

يجب
---------
تومب	-8.1528

تومب
---------
▁ايمكنني	-8.1567

▁ايمكنني
---------
▁تبدو	-8.15951

▁تبدو
---------
مشغول	-8.15957

مشغول
---------
الماضي	-8.15972

الماضي
---------
وافق	-8.16012

وافق
---------
عشر	-8.16117

عشر

---------
ممتع	-8.60754

ممتع
---------
رجع	-8.60905

رجع
---------
▁كانعليك	-8.60935

▁كانعليك
---------
نجح	-8.6094

نجح
---------
كلمة	-8.61004

كلمة
---------
غي	-8.61018

غي
---------
سافر	-8.61092

سافر
---------
نزل	-8.61155

نزل
---------
منالافضل	-8.61281

منالافضل
---------
كرسي	-8.61398

كرسي
---------
▁لماكن	-8.61605

▁لماكن
---------
الراي	-8.61804

الراي
---------
التلفاز	-8.62141

التلفاز
---------
وحده	-8.62208

وحده
---------
الخطة	-8.62361

الخطة
---------
كتابا	-8.62713

كتابا
---------
لميعد	-8.62843

لميعد
---------
رفع	-8.62995

رفع
---------
▁كانذلك	-8.6305

▁كانذلك
---------
متعبا	-8.6362

متعبا
---------
افضلمن	-8.6382

افضلمن
---------
امس	-8.6397

امس
---------
خلال	-8.64022

خلال
---------
نوم	-8.64169

نوم
---------
سير	-8.64205

سير
---------
▁متىت	-8.64503

▁متىت
---------
الاحد	-8.64557

الاحد
---------
يمكننا	-8.64606

يمكننا
---------
راس	-8.64632

راس
---------
لماذا	-8.64821

لماذا
---------
رحل	-8.64895

رحل
---------
جاء	-8.65415

جاء
---------
تان

---------
خيال	-9.30009

خيال
---------
احببت	-9.30058

احببت
---------
نهائي	-9.30107

نهائي
---------
صحة	-9.31221

صحة
---------
قوة	-9.3252

قوة
---------
▁قابلت	-9.34019

▁قابلت
---------
▁حاولتوما	-9.35824

▁حاولتوما
---------
نطق	-9.37511

نطق
---------
مراةال	-9.3782

مراةال
---------
طرق	-9.38765

طرق
---------
نيكيفا	-9.38935

نيكيفا
---------
قادم	-9.39767

قادم
---------
ناقش	-9.42293

ناقش
---------
مستحيل	-9.42373

مستحيل
---------
لنشيدالوطني	-9.42384

لنشيدالوطني
---------
▁اندلعتالحرب	-9.42384

▁اندلعتالحرب
---------
ولاياتالمتحدة	-9.42384

ولاياتالمتحدة
---------
شتاقاليك	-9.42384

شتاقاليك
---------
فييوممنالايام	-9.42384

فييوممنالايام
---------
▁الجوبارد	-9.42384

▁الجوبارد
---------
▁هللياناستعير	-9.42384

▁هللياناستعير
---------
بالتاكيد	-9.42384

بالتاكيد
---------
انبعضكما	-9.42384

انبعضكما
---------
ثانوية	-9.42384

ثانوية
---------
الساعةالثامنة	-9.42384

الساعةالثامنة
---------
الشاطئ	-9.42384

الشاطئ
---------
سيارةاجرة	-9.42384

سيارةاجرة
---------
ممارسة

برازيل
---------
▁قرراني	-10.2072

▁قرراني
---------
مفاتيح	-10.2072

مفاتيح
---------
التفكير	-10.2072

التفكير
---------
ثلاجة	-10.2072

ثلاجة
---------
فيطريقيالىال	-10.2072

فيطريقيالىال
---------
العودةالىال	-10.2072

العودةالىال
---------
قادراعلى	-10.2072

قادراعلى
---------
كنتابحثعنه	-10.2072

كنتابحثعنه
---------
التزلج	-10.2072

التزلج
---------
ونحننلعب	-10.2072

ونحننلعب
---------
اقفلالباب	-10.2072

اقفلالباب
---------
المحتملان	-10.2072

المحتملان
---------
موسيقا	-10.2072

موسيقا
---------
وضوح	-10.2072

وضوح
---------
▁ولدجديعام	-10.2072

▁ولدجديعام
---------
تستخفب	-10.2072

تستخفب
---------
بحالةجيد	-10.2072

بحالةجيد
---------
الامواج	-10.2072

الامواج
---------
مستقيما	-10.2072

مستقيما
---------
تقاسمال	-10.2072

تقاسمال
---------
تنفسيبال	-10.2072

تنفسيبال
---------
تحبالبرتقال	-10.2072

تحبالبرتقال
---------
رفضتعرضه	-10.2073

رفضتعرضه
---------
ماالذيعليفعله	-10.2073

ماالذيعليفعله
---------
حذائي	-10.2073

حذائي
---------
لانيلماك	-10.2073

لانيلماك
---------

In [142]:
ar_vocab_list[0:200]

['<unk>',
 '<s>',
 '</s>',
 'ا',
 '▁',
 'ت',
 'ي',
 'ال',
 '؟',
 'ب',
 'م',
 'ك',
 'ة',
 'ل',
 'ه',
 'ن',
 'في',
 'و',
 'ع',
 '▁ا',
 'توم',
 'من',
 'ها',
 'ف',
 'ان',
 'د',
 'ر',
 'ق',
 'نا',
 'ح',
 'ني',
 'ج',
 'ط',
 'ما',
 'ص',
 'خ',
 'ش',
 'ين',
 'ز',
 'ذلك',
 'على',
 '▁هل',
 'لا',
 'ى',
 'الى',
 'س',
 'انت',
 '!',
 'ون',
 'ية',
 '▁انا',
 'عن',
 'ض',
 'هذا',
 'هنا',
 'لي',
 'الم',
 'غ',
 'ار',
 'ست',
 'ئ',
 'اء',
 'اني',
 'فيال',
 'ته',
 'بال',
 'كل',
 'وا',
 '▁سا',
 '▁لقد',
 'عمل',
 'الان',
 'هم',
 'ث',
 'ذ',
 'بعد',
 'ذهب',
 'انا',
 'ات',
 'هو',
 '▁توم',
 '▁انه',
 '▁انت',
 'تك',
 'عد',
 'منال',
 'جدا',
 '▁هذا',
 'را',
 'مع',
 'كان',
 'سر',
 'ام',
 '▁ال',
 'ور',
 'لل',
 'تحدث',
 'سي',
 'ماري',
 'هي',
 'لك',
 'هذهال',
 'اس',
 '▁كان',
 'فعل',
 '▁لات',
 'كتب',
 'اد',
 'قل',
 'اليوم',
 'علىال',
 'هناك',
 'ابي',
 'سا',
 '"',
 'قد',
 '▁ما',
 'ير',
 'ري',
 'اكل',
 '▁انها',
 'مس',
 '▁اين',
 'احد',
 'اع',
 'غدا',
 'قبل',
 'هذاال',
 '▁هلت',
 'حقا',
 'عرف',
 '▁ان',
 '▁لن',
 'اف',
 'انه',
 '▁م

In [143]:
text_file_en = open("data/en.vocab", "r")
text_file_ar = open("data/msa.vocab", "r")

line = text_file_en.readlines()

In [144]:
en_vocab = Vocab(strings=en_vocab_list)
spacy_en_tokenizer = Tokenizer(en_vocab)

ar_vocab = Vocab(strings=ar_vocab_list)
spacy_msa_tokenizer = Tokenizer(ar_vocab)

## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [145]:
ls data/

AD_NMT-master/
AD_NMT-master.zip
PADIC.xml
UNv1.0.ar-en.tar.gz.00
UNv1.0.ar-en.tar.gz.01
ar_formatted_.txt
ar_formatted_test.txt
ar_formatted_train.txt
ar_formatted_validation.txt
arabic-to-english-translation-sentences.zip
arabic_data.txt
en.model
en.vocab
en_formatted_.txt
en_formatted_test.txt
en_formatted_train.txt
english_data.txt
lav_formatted_.txt
mag_formatted_.txt
msa.model
msa.vocab
msa_formatted_.txt
test/
train/
validation/


In [146]:
msa_tokenizer = T5Tokenizer("data/msa.model")
en_tokenizer = T5Tokenizer("data/en.model")

In [147]:
input_ids = msa_tokenizer.encode('هل بامكاني الدخول؟ </s>', return_tensors='pt')

In [148]:
input_ids

tensor([[ 41,   4, 330,   6,  93, 964,   8,   4,   2]])

In [149]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?

In [150]:
ls data/train

en_formatted_.txt   lav_formatted_.txt  msa_formatted_.txt


## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [151]:
SRC = Field(tokenize = spacy_en_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = spacy_msa_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

In [152]:
dataset = TranslationDataset(path='data/', exts=('en_formatted_.txt', 'msa_formatted_.txt'), fields=(SRC, TRG))

In [153]:
SRC.build_vocab(dataset)
TRG.build_vocab(dataset)

In [154]:
ex = dataset[0]

In [155]:
print(ex.src) 
print(ex.trg)

_0000\01\Tom was also there
_0000\01\يارب لا تدع اهدافهم تصيب حتى العارضة


In [156]:
ls data/train

en_formatted_.txt   lav_formatted_.txt  msa_formatted_.txt


In [164]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = dataset.splits(path= 'data/', exts=('/en_formatted_.txt', '/msa_formatted_.txt'),
                                                    fields = (SRC, TRG))

In [165]:
BATCH_SIZE =32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

NameError: name 'device' is not defined

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

In [ ]:
configuration = T5Config()

In [ ]:
model = T5Model.from_pretrained('t5-base')

In [ ]:
for i, batch in enumerate(train_iterator)

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py